In [1]:
from datetime import timedelta
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

import pandas as pd
from google.cloud import bigquery
from dotenv import load_dotenv

In [2]:
load_dotenv()

client = bigquery.Client()

Create a list of queries with specific dates

In [3]:
start_date = datetime(2017, 1, 1)
end_date = datetime(2017, 1, 4)

current_date = start_date

queries = [
    f"SELECT * "
    f"FROM `bigquery-public-data.google_analytics_sample.ga_sessions_{current_date.strftime('%Y%m%d')}`"
    for current_date in (start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1))
]

Concurrently retrieve data from bigquery with list of queries

In [4]:
def execute_query(query):
    return client.query(query).to_dataframe()


def execute_queries(queries):
    with ThreadPoolExecutor(max_workers=4) as executor:
        results = executor.map(execute_query, queries)

        return list(results)

dataframes = execute_queries(queries)

C:\Users\PC\Desktop\Employment\newage.agency\DanaAnalysTT\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Making dataframe from dataframes list

In [8]:
df = pd.concat(dataframes)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7777 entries, 0 to 2389
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   visitorId             0 non-null      Int64 
 1   visitNumber           7777 non-null   Int64 
 2   visitId               7777 non-null   Int64 
 3   visitStartTime        7777 non-null   Int64 
 4   date                  7777 non-null   object
 5   totals                7777 non-null   object
 6   trafficSource         7777 non-null   object
 7   device                7777 non-null   object
 8   geoNetwork            7777 non-null   object
 9   customDimensions      7777 non-null   object
 10  hits                  7777 non-null   object
 11  fullVisitorId         7777 non-null   object
 12  userId                0 non-null      object
 13  channelGrouping       7777 non-null   object
 14  socialEngagementType  7777 non-null   object
dtypes: Int64(4), object(11)
memory usage: 1002.